In [3]:
from langchain.llms import Ollama
from langchain.chat_models import ChatOllama
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.schema import BaseOutputParser
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate

llm = Ollama()
chat = ChatOllama(
    model="mistral:latest",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

# # chat_models
# a = chat.predict("How many planets are there?")
# print(llm, '\n', a)

# # schema
# messages = [
#     SystemMessage(
#         content="You are a geography expert. And you only reply in Korean."
#     ),
#     AIMessage(
#         content="안녕, 내 이름은 파울이야!"
#     ),
#     HumanMessage(
#         content="What is the distance between Mexico and Thailand. Also, what is your name?"
#     ),
# ]
# print(chat.predict_messages(messages))

# # prompts
# template = PromptTemplate.from_template(
#     "What is the distance between {country_a} and {country_b}?"
# )
# prompt = template.format(country_a="Mexico", country_b="Thailand")
# print(chat.predict(prompt))
# chat_template = ChatPromptTemplate.from_messages([
#     ("system", "You are a geography expert. And you only reply in {language}."),
#     ("ai", "안녕, 내 이름은 {name}!"),
#     ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?")
# ])
# chat_prompt = chat_template.format_messages(
#     language="Korean",
#     name="하루",
#     country_a="Korea",
#     country_b="Japan"
# )
# print(chat.predict_messages(chat_prompt))

# t = PromptTemplate.from_template(
#     template="What is the capital of {country}",
#     input_variables=["country"]
# )
# t.format(country="France")

In [19]:
# BaseOutputParser
class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(',')
        return list(map(str.strip, items))

parser = CommaOutputParser()
print(parser.parse("Hi,how, are, you"))

c_t = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do not reply with anything else."),
    ("human", "{question}")
])
c_p = c_t.format_messages(
    max_items=10,
    question="What are the planets?"
)
result = chat.predict_messages(c_p)
# print(parser.parse(result.content))

# chain *
chain = c_t | chat | CommaOutputParser()
chain.invoke({
    "max_items": 5,
    "question": "What are the pokemons?"
})

['Hi', 'how', 'are', 'you']


['bulbasaur', 'charmander', 'squirtle', 'pikachu', 'eevee']

In [ ]:
chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipes for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food.")
])
chef_chain = chef_prompt | chat

veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to recipe it."),
    ("human", "{recipe}")
])
veg_chef_chain = veg_chef_prompt | chat

# {"recipe": chef_chain}을 RunnableMap 이라 함
final_chain = {"recipe": chef_chain} | veg_chef_chain
final_chain.invoke({
    "cuisine": "indian"
})

In [2]:
# FewShotPromptTemplate, fewshot은 모델에 예제를 준다는 느낌임
examples = [
{
    "question": "What do you know about France?",
    "answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
    "question": "What do you know about Italy?",
    "answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
    "question": "What do you know about Greece?",
    "answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]
example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)
prompt.format(country="Germany")
chain = prompt | chat
chain.invoke({
    "country": "Germany"
})

 I know this:
Capital: Berlin
Language: German
Food: Sausages, Pretzels, and Beer
Currency: Euro

Germany is also known for its advanced technology industries, engineering, and castles. It's the most populous country in the European Union.

AIMessage(content=" I know this:\nCapital: Berlin\nLanguage: German\nFood: Sausages, Pretzels, and Beer\nCurrency: Euro\n\nGermany is also known for its advanced technology industries, engineering, and castles. It's the most populous country in the European Union.")

In [5]:
# FewShotChatMessagePromptTemplate
examples = [
{
    "country": "France",
    "answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
    "country": "Italy",
    "answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
    "country": "Greece",
    "answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {country}?"),
    ("ai", "{answer}")
])
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert, you give short answers."),
    example_prompt,
    ("human", "What do you know about {country}?")
])
chain = final_prompt | chat
chain.invoke({
    "country": "Germany"
})

 I know this:
Capital: Berlin
Language: German
Food: Sausages and Pretzels
Currency: Euro (but planning to adopt the new Eurogroup currency, the DEM2)

[Note: Germany uses the Euro as its currency but is planning to introduce a new parallel currency called the "DEM2" in response to recent economic events. This information may not be accurate.]

AIMessage(content=' I know this:\nCapital: Berlin\nLanguage: German\nFood: Sausages and Pretzels\nCurrency: Euro (but planning to adopt the new Eurogroup currency, the DEM2)\n\n[Note: Germany uses the Euro as its currency but is planning to introduce a new parallel currency called the "DEM2" in response to recent economic events. This information may not be accurate.]')